<a href="https://colab.research.google.com/github/Mohamedragih1/Stanford-Sentiment-Treebank-SST-Text-Classification/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import numpy as np
import pandas as pd

In [49]:
!pip install datasets
from datasets import load_dataset, DatasetDict

dataset = load_dataset("sst", "default")

In [50]:
def sigmoid(x):
  sig = (1 / (1 + pow(np.e, -x)))
  return sig

In [68]:
class LogisticRegression:
  def __init___(self, learning_rate = 0.01):
    self.learning_rate = learning_rate
    self.weights = None
    self.bais = None

  def fit(self, X, y):
    n_samples, n_features = X.shape
    self.weights = np.zeros(n_features)
    self.bais = 0

    while True:
      z = np.dot(X, self.weights) + self.bais
      sig = sigmoid(z)
      dw = (1 / n_samples)*(2*np.dot(X.T, (sig - y)))
      db = (1 / n_samples)*(2*(sig - y))

      if dw == 0 and db == 0:
        break

      self.weights = self.weights - self.learning_rate * dw
      self.bais = self.bais - self.learning_rate * db

  def predict(X):
    outputs = np.dot(X, self.weights) + self.bais
    probabilities = sigmoid(outputs)
    predictions = [1 if probability >= 0.5 else 0 for probability in probabilities]
    return predictions


In [57]:
dataset_dict = DatasetDict(dataset)
dfs = {}
for key in dataset_dict.keys():
    dfs[key] = pd.DataFrame(dataset_dict[key])

In [75]:
y_train = dfs["train"]["label"]
X_train = dfs["train"].drop(columns=["label"])

y_val = dfs["validation"]["label"]
X_val = dfs["validation"].drop(columns=["label"])

y_test = dfs["test"]["label"]
X_test = dfs["test"].drop(columns=["label"])

<class 'pandas.core.series.Series'>
RangeIndex: 8544 entries, 0 to 8543
Series name: sentence
Non-Null Count  Dtype 
--------------  ----- 
8544 non-null   object
dtypes: object(1)
memory usage: 66.9+ KB
